**Section 1: Multiple Regression**

A healthcare non-profit is interested in understanding the impact of statewide demographic information and cigarette prices on cigarette sales. They feel that if any of these factors are significantly related to cigarette sales, it will help them figure out which areas should be targeted with anti-smoking messaging.

    1. Answer the following:
        a. What is the outcome?
        b. What are the predictors they want to understand the impact of?
        c. What is the hypothesis?

    a. We are trying to explain the sales of cigarettes based on a number of predictors, so the outcome is Sales.
    b. The predictors are all of the demographic data and the price of cigarettes. Age, HS, Income, Black, Female, and Price
    c. The hypothesis is that in independent variables influence the total sales of cigarettes. Specifically, I hypothesize that increased price leads to decreased sales, and the other factors influence sales either positively or negatively.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
import statsmodels.api as sm

cig_sales = pd.read_csv("cigarette_sales.csv")
print(cig_sales.head())
cig_sales.describe()

Missing values: all columns have 51 obs, so there are no missing records in the data.

Spread of the data: Income has a broad range compared to the other variables, so we will likely want to scale the data.

Skew: the mean is 10% and the min/max is 0.2/71.1, so the data is very skewed.

Sales: The max is very high compared to the 75% value, so there may be an outlier in Sales.

In [ ]:
sns.pairplot(cig_sales)

Black is very skewed, so we will need to transform this. Sales has a broad spread so we probably want to transform this and find out about potential outliers.

In [ ]:
# transform data
cig_sales['log_sales'] = np.log(cig_sales['Sales'])
cig_sales['log_price'] = np.log(cig_sales['Price'])
cig_sales['log_black'] = np.log(cig_sales['Black'])
cig_sales['log_income'] = np.log(cig_sales['Income'])

sns.pairplot(cig_sales[['log_sales','log_income','log_price','log_black','Age','HS','Female']])

The transformed data appears to be more symmetric and better fits the assumptions of a multiple regression model.

In [ ]:
sns.boxplot(x=cig_sales["log_sales"])
plt.show()

There does appear to be outliers in the Sales data.

In [ ]:
X = cig_sales[['log_income','log_price','log_black','Age','HS','Female']]
y = cig_sales['log_sales']
X = sm.add_constant(X)

model = sm.OLS(y,X)
results = model.fit()
results.summary()


Price and Income are significant, both having P-Values below the alpha, 0.05. The coefficients tell us that as income increases, smoking sales also increases, but as price decreases, sales also decrease. The intercept tells us that with everything else being at 0, the minimum sales is ~0.7, but it is not significant. This is likely because of the log transformation on the variables.

**Section 2: Detecting Assumption Violations**

Using the same data set and regression results from the prior seciont, do the following:

    1. Collinearity
        a. Compute the VIF for each covariate and explain what the results mean.
        b. Compute all the pairwise correlations between the variables.
        c. Remove the 3 variables with the highest p-values. Refit the models, How have the other variables changed? Did R2 change by much?

In [ ]:
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor

y, X = dmatrices('log_sales~log_income+log_price+log_black+Age+HS+Female',data = cig_sales, return_type='dataframe')

vif = pd.DataFrame()
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['variable'] = X.columns

vif

the VIFs here tell us that since log_income, log_black, and HS have high values (higher than 1), there is a moderate correlation between the given explanatory variables and the other explanatory variables. The closer the values are to 5, the more severe they are, but since none of them are higher than five, they likely don't require attention.

In [ ]:
corr = cig_sales[['log_income','log_price','log_black','Age','HS','Female','log_sales']].corr()
sns.heatmap(corr, cmap="coolwarm", annot=True)
plt.title('Correlation Heatmap')
plt.show()

There is a moderate positive correlation between HS and Income and between Female and Black.

There is a moderate negative correlation between Black and Hs and between HS and Female.

In [ ]:
X = cig_sales[['log_income','log_price','Age']]
y = cig_sales['log_sales']
X = sm.add_constant(X)

model = sm.OLS(y,X)
results = model.fit()
results.summary()

The p-values for price and income got slightly higher, but both are still very low and still significant. Age now has a lower p-value and is now significant to the model- higher age predicts higher sales. The constant is also significant now.

The R2 for the model has gotten slightly lower, from 40% to 37%, so less variance is explained by the model, although it is small.

The AIC has gotten lower for the second model, which tells us that the model now better explains the relationships between the variables in the model.

In [ ]:
# fitted values
model_fitted_y = results.fittedvalues

#model residuals
model_residuals = results.resid

# normalized residuals
model_norm_residuals = results.get_influence().resid_studentized_internal

# absolute squared normalized residuals
model_norm_residuals_abs_sqrt = np.sqrt(np.abs(model_norm_residuals))

# absolute residuals
model_abs_resid = np.abs(model_residuals)

# leverage model
model_leverage = results.get_influence().hat_matrix_diag

# cook's distance
model_cooks = results.get_influence().cooks_distance[0]

plt.scatter(model_leverage, model_norm_residuals, alpha=0.5)
sns.regplot(x = model_leverage, y = model_norm_residuals, 
            scatter=False, 
            ci=False, 
            lowess=True,
            line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8})

plt.xlim(0,0.30)
plt.ylim(-3,5)
plt.title('Residuals vs Leverage')
plt.xlabel('Leverage')
plt.ylabel('Standardized Residuals')

# annotations
leverage_top_3 = np.flip(np.argsort(model_cooks),0)[:3]

for i in leverage_top_3:
    plt.annotate(i,
                xy=(model_leverage[i],
                   model_norm_residuals[i]))

# shenanigans for cook's distance contours
def graph(formula, x_range, label=None):
    x = x_range
    y = formula(x)
    plt.plot(x, y, label=label, lw=1, ls = '--', color = 'red')

p = len(results.params)

graph(lambda x: np.sqrt((0.5 * p * (1-x)) / x),
     np.linspace(0.001, 0.300, 50),
     "Cook's distance") # 0.5 line
graph(lambda x: np.sqrt((1 * p * (1-x)) / x),
      np.linspace(0.001, 0.300, 50)) # 1 line

plt.legend(loc='upper right')

There are outliers in the residuals, but it looks as though none of them are outside of cook's distance line, so they don't appear to be influential to the model.

In [ ]:
plt.scatter(model_fitted_y, model_norm_residuals_abs_sqrt, alpha = 0.5)
sns.regplot(x = model_fitted_y, y = model_norm_residuals_abs_sqrt,
           scatter = False,
           ci=False,
           lowess = True,
           line_kws = {'color': 'red', 'lw': 1, 'alpha': 0.8})

plt.title('Scale-Location')
plt.xlabel('Fitted values')
plt.ylabel('$\sqrt{|Standardized Residuals|}$')

# annotations
abs_sq_norm_resid = np.flip(np.argsort(model_norm_residuals_abs_sqrt),0)
abs_sq_norm_resid_top_3 = abs_sq_norm_resid[:3]

for i in abs_sq_norm_resid_top_3:
    plt.annotate(i,
                xy=(model_fitted_y[i],
                    model_norm_residuals_abs_sqrt[i]));

This data likely satisfies the linearity assumption. The line appears to be reasonably flat and the residuals have a fairly uniform spread. There are a few labeled points that appear to be outliers, but we know from the prior analysis that they are not influential to the model.

In [ ]:
from statsmodels.graphics.gofplots import ProbPlot

fig, ax = plt.subplots() 

QQ = ProbPlot(model_norm_residuals)
QQ.qqplot(line='45', alpha=0.5, color='#4C72B0', lw=1, ax=ax)

# Set title and labels on the Axes
ax.set_title('Normal Q-Q')
ax.set_xlabel('Theoretical Quantiles')
ax.set_ylabel('Standardized Residuals')

# annotations
abs_norm_resid = np.flip(np.argsort(np.abs(model_norm_residuals)), 0)
abs_norm_resid_top_3 = abs_norm_resid[:3]

for r, i in enumerate(abs_norm_resid_top_3):
    ax.annotate(i,
                xy=(np.flip(QQ.theoretical_quantiles, 0)[r],
                    model_norm_residuals[i]))

plt.show()

The residuals are approximately normally distributed. There are a few deviations at the tails, but overall the residuals are normal.